In [1]:
from allennlp.commands.elmo import ElmoEmbedder
from pathlib import Path
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import dill
import numpy as np
from matplotlib.ticker import NullFormatter
from sklearn import manifold, datasets
import s3fs
import csv
import torch
import torch.optim as optim
import torch.nn as nn
from torchviz import make_dot

/mnt/efs/home/ctg3039/.conda/envs/allennlp/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [ ]:
import torchviz as viz
model_dir = Path('.')
weights = model_dir / 'weights.hdf5'
options = model_dir / 'options.json'
#For using GPU
seqvec  = ElmoEmbedder(options,weights,cuda_device=0)

In [ ]:
viz.make_dot(seqvec.elmo_bilm.mean())

In [ ]:
##Get the protein sequence
raw_data = pd.read_csv('../CoMET/example/sprot_dna_tf_pfam.tsv', sep="\t", header='infer')
raw_data.columns = raw_data.columns.str.strip().str.lower().str.replace(' ', '_')
length = [len(x) for x in raw_data.sequence]

In [ ]:
#Get the embeddings of the TFs
kernelList = dill.load(open('TF-SeqVec-Embeddings.pickle','rb'))
kList=[]
for entry in raw_data.entry:
    kList.append(kernelList[entry].numpy())

In [2]:
def csv_parser(filename, codes=False, code_key=None, sep='\t'):
    def fam(x):
        dt = str(x).split(',')
        f = [d for d in dt if d.find(' family') >= 0]

        try:
            return f[0]
        except IndexError:
            return "Unassigned"

    def supfam(x):
        dt = str(x).split(',')
        f = [d for d in dt if d.find('superfamily') >= 0]

        try:
            return f[0]
        except IndexError:
            return "Unassigned"

    def subfam(x):
        dt = str(x).split(',')
        f = [d for d in dt if d.find('subfamily') >= 0]

        try:
            return f[0]
        except IndexError:
            return "Unassigned"

    try:
        raw_data = pd.read_hdf(filename.split('.')[0] + '.h5', 'raw_data')
    except FileNotFoundError:
        raw_data = pd.read_csv(filename, sep=sep, header='infer')
        raw_data.columns = raw_data.columns.str.strip().str.lower().str.replace(' ', '_')
        try:
            raw_data['fam'] = raw_data['protein_families'].apply(fam)
            raw_data['sup'] = raw_data['protein_families'].apply(supfam)
            raw_data['sub'] = raw_data['protein_families'].apply(subfam)
        except KeyError:
            pass
        
        raw_data.to_hdf(filename.split('.')[0] + '.h5', 'raw_data')
        
    ##For subsetting the data
    raw_data = raw_data.head(n=2000)
    if codes:
        if type(code_key) == str:
            code_vc = raw_data[code_key].value_counts()
            pos_data = raw_data[raw_data[code_key] != 'Unassigned'][
                ~raw_data[code_key].isin(code_vc[code_vc == 1].index.tolist())]
            code_cats = pos_data[code_key].astype('category').cat.codes
            y_data = code_cats.tolist()
            y_data = [y + 1 for y in y_data]
            x_data = pos_data.sequence
            names = pos_data.entry
        else:
            pf = raw_data[code_key]
            pos_data = raw_data[pf[code_key[0]].notnull() & pf[code_key[1]].notnull()]
            y_data = [pos_data[k].tolist() for k in code_key]
            x_data = pos_data.sequence
            names = pos_data.entry
    else:
        x_data = raw_data.sequence
        y_data = None

    return x_data, y_data, names

x_data, y_data, proteinNames = csv_parser("../CoMET/example/sprot_dna_tf_pfam.tsv", codes=True, code_key="fam")

/mnt/efs/home/ctg3039/.conda/envs/allennlp/lib/python3.6/site-packages/ipykernel_launcher.py:49: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [ ]:
d = pd.DataFrame(zip(proteinNames,y_data))
with open("TFproteinsWithFamilies.txt","w") as handle:
    #writer = csv.writer(handle, delimiter='\t')
    for index, row in d.iterrows():
        handle.write(row[0]+"\t"+str(row[1]))
        handle.write("\n")


In [ ]:
encodingTensors = dill.load(open('TF-SeqVec-Protein-Embeddings.pickle','rb'))
#encodingTensors = dill.load(open('TF-SeqVec-Embeddings.pickle','rb'))

In [ ]:
filteredEncodingTensors = []
for protein in proteinNames:
    filteredEncodingTensors.append(encodingTensors[protein].numpy())

In [ ]:
dill.dump(filteredEncodingTensors1,open('TF-SeqVec-Protein-WithFamilies-Embeddings.pickle', 'wb'))

In [ ]:
import torch
from torch import nn

a = torch.randn(32, 100, 1)  
m = nn.Conv1d(100, 100, 1) 
out = m(a)
print(out.size())
print(m)

In [ ]:
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader
torch.manual_seed(42)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
x_tensor = torch.stack(filteredEncodingTensors)
#x_tensor = filteredEncodingTensors
y_tensor = torch.from_numpy(np.array(y_data)).float()

class CustomDataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        self.x = x_tensor
        self.y = y_tensor
        
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.x)

# Builds dataset with ALL data
dataset = CustomDataset(x_tensor, y_tensor)

# Splits randomly into train and validation datasets
train_dataset, val_dataset = random_split(dataset, [5615, 992])

# Builds a loader for each dataset to perform mini-batch gradient descent
train_loader = DataLoader(dataset=train_dataset, batch_size=20)
val_loader = DataLoader(dataset=val_dataset, batch_size=20)

model1 = nn.Sequential(
            nn.Conv1d(1, 128, kernel_size=5,stride=1),
            nn.ReLU(inplace=True),
            #nn.MaxPool1d(kernel_size=pool_kernel_size, stride=pool_kernel_size),
            nn.Dropout(p=0.2),

            #nn.Conv1d(128, 256, kernel_size=5,stride=1),
            #nn.ReLU(inplace=True),
            #nn.MaxPool1d(kernel_size=pool_kernel_size, stride=pool_kernel_size),
            nn.Dropout(p=0.2),
            nn.Linear(128,256),
            #nn.ReLU(inplace=True),
            nn.Linear(256,240),
            nn.Sigmoid()).to(device)
print(model1)

# Sets hyper-parameters
lr = 1e-1
n_epochs = 10

# Defines loss function and optimizer
loss_fn = nn.MSELoss(reduction='mean')
optimizer = optim.SGD(model1.parameters(), lr=lr)

losses = []
val_losses = []

# Creates function to perform train step from model, loss and optimizer
def make_train_step(model, loss_fn, optimizer):
    # Builds function that performs a step in the train loop
    def train_step(x, y):
        # Sets model to TRAIN mode
        model.train()
        # Makes predictions
        yhat = model(x)
        # Computes loss
        loss = loss_fn(y, yhat)
        # Computes gradients
        loss.backward()
        # Updates parameters and zeroes gradients
        optimizer.step()
        optimizer.zero_grad()
        # Returns the loss
        return loss.item()
    
    # Returns the function that will be called inside the train loop
    return train_step
train_step = make_train_step(model1, loss_fn, optimizer)

In [ ]:
# Training loop
for epoch in range(n_epochs):
    # Uses loader to fetch one mini-batch for training
    for x_batch, y_batch in train_loader:
        # NOW, sends the mini-batch data to the device
        # so it matches location of the MODEL
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        # One stpe of training
        loss = train_step(x_batch, y_batch)
        losses.append(loss)
        
    # After finishing training steps for all mini-batches,
    # it is time for evaluation!
        
    # We tell PyTorch to NOT use autograd...
    # Do you remember why?
    with torch.no_grad():
        # Uses loader to fetch one mini-batch for validation
        for x_val, y_val in val_loader:
            # Again, sends data to same device as model
            x_val = x_val.to(device)
            y_val = y_val.to(device)
            
            # What is that?!
            model.eval()
            # Makes predictions
            yhat = model(x_val)
            # Computes validation loss
            val_loss = loss_fn(y_val, yhat)
            val_losses.append(val_loss.item())

In [ ]:
torch.manual_seed(42)

#x_tensor = torch.from_numpy(x).float()
x_tensor = filteredEncodingTensors
y_tensor = torch.from_numpy(y_data).float()

# Builds dataset with ALL data
dataset = TensorDataset(x_tensor, y_tensor)

# Splits randomly into train and validation datasets
train_dataset, val_dataset = random_split(dataset, [80, 20])

# Builds a loader for each dataset to perform mini-batch gradient descent
train_loader = DataLoader(dataset=train_dataset, batch_size=16)
val_loader = DataLoader(dataset=val_dataset, batch_size=20)

# Builds a simple sequential model
model = nn.Sequential(nn.Linear(1, 1)).to(device)
print(model.state_dict())
model1 = nn.Sequential(
            nn.Conv1d(1, 128, kernel_size=conv_kernel_size),
            nn.ReLU(inplace=True),
            #nn.MaxPool1d(kernel_size=pool_kernel_size, stride=pool_kernel_size),
            nn.Dropout(p=0.2),

            nn.Conv1d(128, 256, kernel_size=conv_kernel_size),
            nn.ReLU(inplace=True),
            #nn.MaxPool1d(kernel_size=pool_kernel_size, stride=pool_kernel_size),
            nn.Dropout(p=0.2),
            #nn.Linear(960 * self.n_channels, n_genomic_features),
            #nn.ReLU(inplace=True),
            nn.Linear(256,50),
            nn.Sigmoid())


# Sets hyper-parameters
lr = 1e-1
n_epochs = 150

# Defines loss function and optimizer
loss_fn = nn.MSELoss(reduction='mean')
optimizer = optim.SGD(model.parameters(), lr=lr)

losses = []
val_losses = []

# Creates function to perform train step from model, loss and optimizer
def make_train_step(model, loss_fn, optimizer):
    # Builds function that performs a step in the train loop
    def train_step(x, y):
        # Sets model to TRAIN mode
        model.train()
        # Makes predictions
        yhat = model(x)
        # Computes loss
        loss = loss_fn(y, yhat)
        # Computes gradients
        loss.backward()
        # Updates parameters and zeroes gradients
        optimizer.step()
        optimizer.zero_grad()
        # Returns the loss
        return loss.item()
    
    # Returns the function that will be called inside the train loop
    return train_step
train_step = make_train_step(model, loss_fn, optimizer)
# Training loop
for epoch in range(n_epochs):
    # Uses loader to fetch one mini-batch for training
    for x_batch, y_batch in train_loader:
        # NOW, sends the mini-batch data to the device
        # so it matches location of the MODEL
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        # One stpe of training
        loss = train_step(x_batch, y_batch)
        losses.append(loss)
        
    # After finishing training steps for all mini-batches,
    # it is time for evaluation!
        
    # We tell PyTorch to NOT use autograd...
    # Do you remember why?
    with torch.no_grad():
        # Uses loader to fetch one mini-batch for validation
        for x_val, y_val in val_loader:
            # Again, sends data to same device as model
            x_val = x_val.to(device)
            y_val = y_val.to(device)
            
            # What is that?!
            model.eval()
            # Makes predictions
            yhat = model(x_val)
            # Computes validation loss
            val_loss = loss_fn(y_val, yhat)
            val_losses.append(val_loss.item())

print(model.state_dict())
print(np.mean(losses))
print(np.mean(val_losses))

In [16]:
import keras.backend as K
from absl import flags
from keras import regularizers
from keras.layers import Activation, BatchNormalization, Convolution1D, Dense, Flatten, GlobalMaxPooling1D, Input, \
    MaxPooling1D, Reshape
from keras.losses import binary_crossentropy, categorical_crossentropy
from keras.metrics import binary_accuracy, categorical_accuracy
from tensorflow.python.ops.nn_ops import conv1d_transpose
from keras.utils.np_utils import to_categorical
from evolutron.engine import Model, load_model
from evolutron.templates import callback_templates as cb

##Tensorflow Model
def build_cofam_model(input_shape=None, output_dim=None, saved_model=None):
    if saved_model:
        model = load_model(saved_model, custom_objects=custom_layers, compile=False)
        model.classification = True
    else:
        seq_length, alphabet = input_shape
        # Model Architecture
        # Input LayerRO
        inp = Input(shape=input_shape, name='ProteinEmbeddings')
        feature_layer = inp
        feature_layer = Convolution1D(filters=128,
                                          kernel_size=50,
                                          strides=1,
                                          padding='same',
                                          use_bias=False,
                                          kernel_initializer='glorot_uniform',
                                          activation='linear',
                                          name="ConvLayer1")(feature_layer)
        #feature_layer = BatchNormalization()(feature_layer)
        feature_layer = Activation(activation='relu',name="ActivationLayer1")(feature_layer)
        
        feature_layer = Convolution1D(filters=256,
                                          kernel_size=50,
                                          strides=1,
                                          padding='same',
                                          use_bias=False,
                                          kernel_initializer='glorot_uniform',
                                          activation='linear',
                                          name="ConvLayer2")(feature_layer)
        #feature_layer = BatchNormalization()(feature_layer)
        feature_layer = Activation(activation='relu',name="ActivationLayer2")(feature_layer)

        # Max-pooling
        if seq_length:
            max_pool = MaxPooling1D(pool_size=seq_length,name="Max-Pooling")(feature_layer)
            flat = Flatten(name="Flatten")(max_pool)
        else:
            # max_pool = GlobalMaxPooling1D()(convs[-1])
            # flat = max_pool
            raise NotImplementedError('Sequence length must be known at this point. Pad and use mask.')

        # Fully-Connected encoding layers
        fc_enc = Dense(256,
                        kernel_initializer='glorot_uniform',
                        activation='relu',
                        name='FullyConnected1')(flat)

        encoded = Dense(256,
                        kernel_initializer='glorot_uniform',
                        activation='relu',
                        name='FullyConnected2')(fc_enc)

        classifier = Dense(output_dim,
                           kernel_initializer='glorot_uniform',
                           activation='softmax',
                           name='Classifier')(encoded)

        model = Model(inputs=inp, outputs=classifier, name='DeepClassfier', classification=True)

    # Loss Functions
    losses = [categorical_crossentropy]

    # Metrics
    metrics = [categorical_accuracy]

    # Compilation

    model.compile(optimizer="nadam",
                  loss=losses,
                  metrics=metrics,
                  lr=0.002)
    return model



In [4]:
filteredEncodingTensors = dill.load(open('TF-SeqVec-Protein-WithFamiliesTop2000-Embeddings.pickle','rb'))

In [ ]:
filteredEncodingTensors = []
for protein in proteinNames:
    filteredEncodingTensors.append(encodingTensors[protein].numpy())

In [5]:
##Padding of the dataset
def pad_or_clip_seq(x, n):
    if n >= x.shape[0]:
        b = np.zeros((n, x.shape[1]))
        b[:x.shape[0]] = x
        return b
    else:
        return x[:n, :]
    
def preprocess_dataset(x_data, y_data=None, one_hot='x', padded=True, pad_y_data=False, nb_aa=20, min_aa=None,
                       max_aa=None):
    """

    Args:
        x_data (pd.Series):
        y_data (list or np.ndArray):
        one_hot (str):
        padded (bool):
        pad_y_data (bool):
        nb_aa:
        min_aa:
        max_aa:

    Returns:

    """
    if not max_aa:
        max_aa = int(np.percentile([len(x) for x in x_data], 99))  # pad so that 99% of datapoints are complete
    else:
        max_aa = min(max_aa, np.max([len(x) for x in x_data]))
    x_data = np.asarray([pad_or_clip_seq(x, max_aa) for x in x_data], dtype=np.float32)

    y_data = np.asarray(y_data)
    assert ((len(x_data) == len(y_data)) or (len(x_data) == len(y_data[0])))
    data_size = len(x_data)
    print('Dataset size: {0}'.format(data_size))
    return x_data, y_data

In [6]:
x_data,y_data=preprocess_dataset(filteredEncodingTensors,y_data,max_aa=1353)

Dataset size: 1094


In [17]:
#x_data,y_data=preprocess_dataset(filteredEncodingTensors,y_data)

input_shape = x_data[0].shape
y_data = to_categorical(y_data)
output_dim = y_data.shape[1]
conv_net = build_cofam_model(input_shape,output_dim)
conv_net.display_network_info()

____________________________________________________________________________________________________
Layer (type)                                 Output Shape                            Param #        
ProteinEmbeddings (InputLayer)               (None, 1353, 1024)                      0              
____________________________________________________________________________________________________
ConvLayer1 (Conv1D)                          (None, 1353, 128)                       6553600        
____________________________________________________________________________________________________
ActivationLayer1 (Activation)                (None, 1353, 128)                       0              
____________________________________________________________________________________________________
ConvLayer2 (Conv1D)                          (None, 1353, 256)                       1638400        
___________________________________________________________________________________________

In [ ]:
callbacks = cb.standard(patience=20, reduce_factor=0.5)
#print('Started training at {}'.format(time.asctime()))
conv_net.fit(x_data, y_data,epochs=10,batch_size=64,validation_split=0.15,callbacks=callbacks)

In [18]:
from keras.utils import plot_model
plot_model(conv_net, to_file='model.png')